<a href="https://colab.research.google.com/github/ThalitaCoelhoAguiar/meus-notebooks-colab/blob/main/IEEE_4PERIODO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabalho Tópicos
### Thalita Coelho Aguiar/ 222030324


Objetivo: Determinar a espécie de uma muda a partir de uma imagem
Utilizado: CNN simples com framework Keras


In [104]:
# Importando bibliotecas
import numpy as np

from google.colab import files
import torch #Importa a biblioteca PyTorch
from torchvision import datasets, transforms # aplicar pré-processamento

import torch.nn as nn # Importa o módulo de redes neurais
import torch.nn.functional as F # Importa as funções

import torch.optim as optim
import matplotlib.pyplot as plt

!pip install torchsummary
from torchvision import datasets, transforms



In [127]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

### Importando o Dataset

Dataset consiste em e mudas de plantas e contém imagens de aproximadamente 960 plantas
únicas pertencentes a 12 espécies em vários estágios de crescimento. Ele compreende imagens RGB anotadas
com uma resolução física de aproximadamente 10 pixels por mm.


*   Importando diretamente do Kaggle
* Escolher arquivo kaggle.json



In [30]:
uploaded = files.upload()

Saving classificandos-nuvens-usando-cnn.zip to classificandos-nuvens-usando-cnn (1).zip


In [55]:
!unzip -o classificandos-nuvens-usando-cnn.zip

Archive:  classificandos-nuvens-usando-cnn.zip
  inflating: clouds/clouds_test/cirriform clouds/0208a6357f23cf53980f72dea42ade63.jpg  
  inflating: clouds/clouds_test/cirriform clouds/03775c45f7cc00d8aa7eccdb482b76af.jpg  
  inflating: clouds/clouds_test/cirriform clouds/04649cdcf0f8bc537233cdb724fee004.jpg  
  inflating: clouds/clouds_test/cirriform clouds/05817a2a04bb3c19684e39adbd0c6583.jpg  
  inflating: clouds/clouds_test/cirriform clouds/11595750ac2dd8fbce2f67ea40d21acd.jpg  
  inflating: clouds/clouds_test/cirriform clouds/18805becb7894a8f8ba128dcf9b99af6.jpg  
  inflating: clouds/clouds_test/cirriform clouds/2426a63aa874c3484a6f298077f342f5.jpg  
  inflating: clouds/clouds_test/cirriform clouds/311b601896e1990fd32cdcf6bd02c0d9.jpg  
  inflating: clouds/clouds_test/cirriform clouds/3267b238863e21c376f41503d971f24c.jpg  
  inflating: clouds/clouds_test/cirriform clouds/340555680976ad462eae36ec7ca31ec4.jpg  
  inflating: clouds/clouds_test/cirriform clouds/3605c63de44e6ecdd029e27e

### Classes

In [59]:
!ls clouds/clouds_train

'cirriform clouds'     'cumulus clouds'		 'stratocumulus clouds'
'clear sky'	       'high cumuliform clouds'
'cumulonimbus clouds'  'stratiform clouds'


##Conversão e normalização

In [71]:
# Transformação básica
transform = transforms.Compose([
    transforms.ToTensor(),          # Converte a imagem para tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza entre [-1, 1]
])

transform = transforms.Compose([
    transforms.ToTensor(),           # Converte para Tensor (mantém 3 canais)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza entre [-1, 1]
])


## Dataset teste e treino

In [72]:
# Dataset de teste
train_dataset = datasets.ImageFolder(
    root='clouds/clouds_train',
    transform=transform)

# Dataset de teste
test_dataset = datasets.ImageFolder(
    root='clouds/clouds_test',
    transform=transform)

### Tamanho da imagem

In [73]:
# Pega um batch de imagens
images, labels = next(iter(train_dataset)) # Using train_set as the dataloader

# Vê o shape das imagens
print(images.shape) #torch.Size([batch_size, canais, altura, largura])

torch.Size([3, 256, 256])


Tamanho da imagem: 256 × 256 pixels.

Canais: 3 (escala RGB)

## Arquitetura

In [92]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 3 canais RGB
        self.pool = nn.MaxPool2d(2, 2)  # 256x256 -> 128x128
        # Cálculo: 32 filtros, 128x128
        self.fc1 = nn.Linear(32 * 128 * 128, 64)  # Corrigido o tamanho!
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### dados do Treinamento e Teste

In [93]:
train_set = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
test_set = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True)

### Treinamento

In [102]:
network = CNN()

optimizer = optim.Adam(network.parameters(), lr=0.001)
epochs = 4

for epoch in range(epochs):
    for data in train_set:
        image, output = data
        network.zero_grad()

        result = network(image)

        loss = F.cross_entropy(result, output)
        loss.backward()
        optimizer.step()
        learn_rate.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')


Epoch 1, Loss: 1.0982
Epoch 2, Loss: 1.3381
Epoch 3, Loss: 0.7696
Epoch 4, Loss: 0.2158


## Treinamento por RESNET

*   


##Treinamento

### Definindo arquitetura



O otimizador adam  é um bom otimizador

A função perda categorical_crossentropy é adequado para a classificação multiclasse imagens.

In [124]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,       # Não inclui as camadas fully connected
    input_shape=(256, 256, 3) # Tamanho da sua imagem
)

# Para ver as camadas
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 262, 262,  │          0 │ input_layer_9[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 128, 128,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 128, 128,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 128, 128,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 130, 130,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 64, 64,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 64, 64,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 64, 64,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 64, 64,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 64, 64,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 64, 64,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 64, 64,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 64, 64,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 64, 64,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

## Criação de CNN

In [132]:
model = Sequential([
    # Primeira camada convolucional
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)), # Alterado o input_shape para 150x150
    MaxPooling2D(pool_size=(2, 2)), #Reduz a dimensionalidade das imagens
    Dropout(0.2),  #desativa 20% das ativações de forma aleatória durante o treinamento.

    Conv2D(64, (3, 3), activation='relu'), # Segunda camada convolucional
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'), # Terceira camada convolucional
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(256, (3, 3), activation='relu'), # Quarta camada convolucional
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),


    Flatten(), # Achatar as saídas das camadas convolucionais

    Dense(256, activation='relu'), # Camada totalmente conectada
    Dropout(0.5), # Corrigido o valor do dropout para 0.5 (deve ser um float entre 0 e 1)

    Dense(12, activation='softmax')  # Camada de saída para classificação das 12 classes
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Treinamento

In [133]:
# Compilar o modelo para o treinamentos ou teste
model.compile(
    optimizer='adam', #ot imizador para minimizar a função perda e ajustar os pesos do modelo
    loss='categorical_crossentropy',#função perda
    metrics=['accuracy']
)